# **ENGG680 - Introduction to Digital Engineering**
## *Lab Assignment 4: ML Applications - Vehicle Motion State Classification*

## Preliminary: Certificate of Work


*We, the undersigned, certify that this is our own work, which has been done expressly for this course, either without the assistance of any other party or where appropriate we have acknowledged the work of others. Further, we have read and understood the section in the university calendar on plagiarism/cheating/other academic misconduct and we are aware of the implications thereof. We request that the total mark for this assignment be distributed as follows among group members:*

|          | First Name | Last Name | Signature (Full Name, Date) | Hours | Contribution % |
|----------|------------|-----------|-----------------------------|-------|----------------|
| Member 1: | First Name | Last Name | Signature | Hours | Contribution |
| Member 2: | First Name | Last Name | Signature | Hours | Contribution |
| Member 3: | First Name | Last Name | Signature | Hours | Contribution |
| Member 4: | First Name | Last Name | Signature | Hours | Contribution |


In [80]:
import pandas as pd

# Define your data
data = {
    "First Name": ["Chan Nyein", "Soheil", "Seyedehshima", "Zohreh"],
    "Last Name": ["Aung", "Seifi", "Moradi", "Mejrisazantoosi"],
    "Signature (Full Name, Date)": ["Chan Nyein Aung, 2024-11-29", "Soheil Seifi, 2024-11-29",
                                     "Seyedehshima Moradi, 2024-11-29", "Zohreh Mejrisazantoosi, 2024-11-29"],
    "Hours": [20, 20, 20, 20],
    "Contribution %": [25, 25, 25, 25]
}

# Create the DataFrame
df = pd.DataFrame(data)
df.index = [f"Member {i+1}" for i in range(len(df))]

# Display the DataFrame
df

,First Name,Last Name,"Signature (Full Name, Date)",Hours,Contribution %
Member 1,Chan Nyein,Aung,"Chan Nyein Aung, 2024-11-29",20,25
Member 2,Soheil,Seifi,"Soheil Seifi, 2024-11-29",20,25
Member 3,Seyedehshima,Moradi,"Seyedehshima Moradi, 2024-11-29",20,25
Member 4,Zohreh,Mejrisazantoosi,"Zohreh Mejrisazantoosi, 2024-11-29",20,25


---

## Section 1 - Motion Classification Using *All Features* Derived From Inertial Sensor Data (33 Marks)

---

### 1.1 Assignment datasets import (6 Marks)

---

In [81]:
# some library imports
import pandas as pd



First load the following datasets into dataframes.
* *UrbanNav-HK_Whampoa-20210521_sensors.csv* to dataframe variable named *cv_set*. This dataset is to be used for model training and hyper parameter tuning.
* *UrbanNav-HK_TST-20210517_sensors.csv* to dataframe variable named *test_set1*. This is first independent test dataset.
* *2020-03-14-16-45-35.csv* to dataframe variable named *test_set2*. This is the second independent test dataset.

In [82]:
# (2 marks)
# TODO: Load cross validation/training dataset (UrbanNav-HK_Whampoa-20210521_sensors.csv) to 'cv_set' dataframe

# uncomment and compelete below
cv_set = pd.read_csv('/content/UrbanNav-HK_Whampoa-20210521_sensors.csv')

# TODO: Load UrbanNav-HK_TST-20210517_sensors.csv to 'test_set1' dataframe
test_set1 = pd.read_csv('/content/UrbanNav-HK_TST-20210517_sensors.csv')

# TODO: Load 2020-03-14-16-45-35.csv to 'test_set2' dataframe
test_set2 = pd.read_csv('/content/2020-03-14-16-45-35.csv')

For the *cv_set*, look at the dataframe head and info, and print the list of all columns. The *test_set1* and *test_set2* have the same schema as *cv_set*, however, have different number of rows (samples).

In [83]:
# TODO: Display first two rows of cv_set
# (1 mark)
print("First two rows of cv_set:")
print(cv_set.head(2))

First two rows of cv_set:
     ax_sum    ay_sum    az_sum    gx_sum    gy_sum    gz_sum     a_sum  \
0 -0.003929 -0.093081  9.789349 -0.002649  0.001406 -0.001391  9.797418   
1 -0.000966 -0.092417  9.789095 -0.001806  0.000103 -0.001943  9.798815   

      g_sum        ax_var        ay_var  ...     g_min    ax_max    ay_max  \
0  0.022554  8.046094e-07  1.319665e-07  ...  0.000003  0.001496  0.000544   
1  0.021731  1.032619e-06  1.073067e-07  ...  0.000006  0.001865  0.000615   

     az_max    gx_max    gy_max    gz_max     a_max     g_max  motion_state  
0  0.024992  0.000110  0.000075  0.000078  0.024997  0.000187          rest  
1  0.024828  0.000124  0.000098  0.000090  0.024829  0.000144          rest  

[2 rows x 33 columns]


In [84]:
# TODO: Load the info method of cv_set
# (1 mark)
print("\nInfo about cv_set:")
print(cv_set.info())


Info about cv_set:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ax_sum        1534 non-null   float64
 1   ay_sum        1534 non-null   float64
 2   az_sum        1534 non-null   float64
 3   gx_sum        1534 non-null   float64
 4   gy_sum        1534 non-null   float64
 5   gz_sum        1534 non-null   float64
 6   a_sum         1534 non-null   float64
 7   g_sum         1534 non-null   float64
 8   ax_var        1534 non-null   float64
 9   ay_var        1534 non-null   float64
 10  az_var        1534 non-null   float64
 11  gx_var        1534 non-null   float64
 12  gy_var        1534 non-null   float64
 13  gz_var        1534 non-null   float64
 14  a_var         1534 non-null   float64
 15  g_var         1534 non-null   float64
 16  ax_min        1534 non-null   float64
 17  ay_min        1534 non-null   float64
 18  az_min  

In [85]:
# TODO: Display the number of rows and columns  of all three datasets
# (1 mark)
print(f"'cv_set' has {cv_set.shape[0]} rows and {cv_set.shape[1]} columns.")
print(f"'test_set1' has {test_set1.shape[0]} rows and {test_set1.shape[1]} columns.")
print(f"'test_set2' has {test_set2.shape[0]} rows and {test_set2.shape[1]} columns.")

'cv_set' has 1534 rows and 33 columns.
'test_set1' has 783 rows and 33 columns.
'test_set2' has 297 rows and 33 columns.


In [86]:
# TODO: Display all columns in cv_set
print("\nColumns in cv_set:")
print(cv_set.columns.tolist())


Columns in cv_set:
['ax_sum', 'ay_sum', 'az_sum', 'gx_sum', 'gy_sum', 'gz_sum', 'a_sum', 'g_sum', 'ax_var', 'ay_var', 'az_var', 'gx_var', 'gy_var', 'gz_var', 'a_var', 'g_var', 'ax_min', 'ay_min', 'az_min', 'gx_min', 'gy_min', 'gz_min', 'a_min', 'g_min', 'ax_max', 'ay_max', 'az_max', 'gx_max', 'gy_max', 'gz_max', 'a_max', 'g_max', 'motion_state']


In all three dataframes, target vector is stored in the *motion_state* column. All other columns are features.

___

### 1.2 - Grid Search with Cross Validation (18 Marks)

___

#### 1.2.1 - Extract all features and target from cv_set

___

From the *cv_set*, load the target vector stored in the *motion_state* column into a new variable *y*

In [87]:
# TODO - Load target vector into y
# (1 mark)
y = cv_set['motion_state']
print("Target vector (y):")
print(y.head())

Target vector (y):
0        rest
1        rest
2    straight
3        turn
4        turn
Name: motion_state, dtype: object


From *cv_set*, load all features, that is all columns except for the target vector, in a new dataframe variable *X*

In [88]:
# TODO - Load all features from cv_set to X
# (2 marks)
X = cv_set.drop(columns=['motion_state'])
print("\nFeature matrix (X):")
print(X.head(2))


Feature matrix (X):
     ax_sum    ay_sum    az_sum    gx_sum    gy_sum    gz_sum     a_sum  \
0 -0.003929 -0.093081  9.789349 -0.002649  0.001406 -0.001391  9.797418   
1 -0.000966 -0.092417  9.789095 -0.001806  0.000103 -0.001943  9.798815   

      g_sum        ax_var        ay_var  ...     a_min     g_min    ax_max  \
0  0.022554  8.046094e-07  1.319665e-07  ...  0.023871  0.000003  0.001496   
1  0.021731  1.032619e-06  1.073067e-07  ...  0.024153  0.000006  0.001865   

     ay_max    az_max    gx_max    gy_max    gz_max     a_max     g_max  
0  0.000544  0.024992  0.000110  0.000075  0.000078  0.024997  0.000187  
1  0.000615  0.024828  0.000124  0.000098  0.000090  0.024829  0.000144  

[2 rows x 32 columns]


___

#### 1.2.2 - Configure and Perform Grid Search with Cross-Validation

___

In [89]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV

The dataset *(X, y)*, must be split into training/validation *(X_train, y_train)* and testing *(X_test, y_test)* sets. Use the `train_test_split` function from sklearn to create the split, where  **30%** of data is used for testing. Set the `random_state=42`, and `stratify=y`.  

In [90]:
# TODO - Perform test/train split
# (1 mark)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Training and testing split done.")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training and testing split done.
Training set: 1073 samples
Testing set: 461 samples


Initialize a `Pipeline`, consisting of a *scaler* and a *classifer*, set the default values to *StandardScaler()*, and *SVC()*, respectively.

In [91]:
# TODO - Initialize Pipeline into variable pipe
# (2 marks)
pipe = Pipeline([
    ('scaler', StandardScaler()),  # Scaling features
    ('classifier', SVC())          # Support Vector Classifier
])

print("Pipeline initialized.")

Pipeline initialized.


Define the grid search configuration (*param_grid*) for training and hyper parameter tuning of the *support vector classifier*. Tune, the hyper-parameter *C*, over the interval `C=[0.01, 0.1, 1 ,10,100]`, *gamma*, over the interval `gamma=[0.001, 0.01, 0.1,1]`, and use the *radial basis function* as the kernel. For the *scaler* parameter, test with the `StandardScaler`, `RobustScaler`, and `None` options.

In [92]:
# TODO - Setup grid set
# (4 marks)

param_grid = {
    'scaler': [StandardScaler(), RobustScaler(), None],  # Test different scalers
    'classifier__C': [0.01, 0.1, 1, 10, 100],           # Regularization parameter
    'classifier__gamma': [0.001, 0.01, 0.1, 1],         # Kernel coefficient for RBF
    'classifier__kernel': ['rbf']                       # Use radial basis function (RBF) kernel
}

print("Parameter grid defined.")

Parameter grid defined.


In [93]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'classifier__kernel': ['rbf'],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 1.2.3 - Visualize Grid Search Results

___

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [94]:
# TODO: Print the results from the grid search
# (2 marks)

# Print the best parameters from the grid search
print("Best parameters:\n", grid.best_params_)

# Best training score from cross-validation
best_train_score = max(grid.cv_results_['mean_train_score'])
print(f"Best cross-validation train score: {best_train_score:.2f}")

# Best validation score from cross-validation
best_val_score = grid.best_score_
print(f"Best cross-validation validation score: {best_val_score:.2f}")

# Test set accuracy
from sklearn.metrics import accuracy_score
y_test_pred = grid.best_estimator_.predict(X_test)
test_score = accuracy_score(y_test, y_test_pred)
print(f"Test-set score: {test_score:.2f}")

Best parameters:
 {'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}
Best cross-validation train score: 1.00
Best cross-validation validation score: 0.95
Test-set score: 0.96


Display the confusion matrix and print the classification report for the trained model. Use the *(X_test, y_test)* dataset. **Hint** - You can use your code from assignment 3.

In [95]:
# TODO - Display confusion matrix and print classification report for (X_test, y_test)
from sklearn.metrics import confusion_matrix, classification_report

# Generate predictions for the test set
y_test_pred = grid.best_estimator_.predict(X_test)

# Display the confusion matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Display the classification report
class_report = classification_report(y_test, y_test_pred)
print("\nClassification Report:")
print(class_report)


Confusion Matrix:
[[153   2   0]
 [ 10 192   5]
 [  0   3  96]]

Classification Report:
              precision    recall  f1-score   support

        rest       0.94      0.99      0.96       155
    straight       0.97      0.93      0.95       207
        turn       0.95      0.97      0.96        99

    accuracy                           0.96       461
   macro avg       0.95      0.96      0.96       461
weighted avg       0.96      0.96      0.96       461



___

#### 1.2.4 - Questions

1. Comment on the fit of the model to the dataset? (2 marks)
2. In the grid search, are the intervals used for *C* and *gamma* hyperparameters suitable? Why or why not? (2 marks)

#### 1.2.4 - Answers
1.  The model fits the dataset very well, achieving a 96% test-set accuracy. This indicates a strong generalization ability. The high F1-scores across all classes show the model is robust and makes minimal misclassifications.
2.  Yes, the intervals for C ([0.01, 0.1, 1, 10, 100]) and gamma ([0.001, 0.01, 0.1, 1]) are suitable. They cover a wide range, allowing the model to find an optimal balance between underfitting and overfitting. The chosen C=100 and gamma=0.01 fall within the tested range, confirming the suitability of these intervals.

___

### 1.3 - Model Evaluation on Test Datasets 1 and 2 (9 Marks)

___

#### 1.3.1 - Extract all features and targets from test datasets

Next, the model trained in 1.2 will be tested on independent datasets, that is *test_set1* and *test_set2*. <br>
The test_sets must be seperated into features (Xs) and targets (ys).

In [96]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# (1 marks)
y1 = test_set1['motion_state']  # Target vector
X1 = test_set1.drop(columns=['motion_state'])  # Feature matrix
print("Test Set 1 Target Vector (y1):")
print(y1.head())
print("\nTest Set 1 Feature Matrix (X1):")
print(X1.head(2))

Test Set 1 Target Vector (y1):
0    rest
1    rest
2    rest
3    rest
4    rest
Name: motion_state, dtype: object

Test Set 1 Feature Matrix (X1):
     ax_sum    ay_sum    az_sum    gx_sum    gy_sum    gz_sum     a_sum  \
0  0.264465  0.089904  9.786930 -0.002305  0.000218 -0.001143  9.801661   
1  0.264300  0.088882  9.788127 -0.001865 -0.000523 -0.001999  9.802481   

      g_sum    ax_var        ay_var  ...     a_min     g_min    ax_max  \
0  0.025634  0.000001  7.749919e-08  ...  0.024056  0.000003  0.002506   
1  0.025218  0.000001  9.243000e-08  ...  0.024084  0.000007  0.002427   

     ay_max    az_max    gx_max    gy_max    gz_max     a_max     g_max  
0  0.000845  0.024884  0.000155  0.000078  0.000069  0.024886  0.000170  
1  0.000949  0.024881  0.000201  0.000095  0.000076  0.024883  0.000201  

[2 rows x 32 columns]


In [97]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
y2 = test_set2['motion_state']  # Target vector
X2 = test_set2.drop(columns=['motion_state'])  # Feature matrix
print("\nTest Set 2 Target Vector (y2):")
print(y2.head())
print("\nTest Set 2 Feature Matrix (X2):")
print(X2.head(2))


Test Set 2 Target Vector (y2):
0    rest
1    rest
2    rest
3    rest
4    rest
Name: motion_state, dtype: object

Test Set 2 Feature Matrix (X2):
     ax_sum    ay_sum    az_sum    gx_sum    gy_sum    gz_sum     a_sum  \
0 -0.046400  0.305907  9.791451 -0.002184  0.001311 -0.001901  9.800683   
1 -0.050073  0.311947  9.785619 -0.003369  0.001898 -0.002051  9.797741   

      g_sum    ax_var        ay_var  ...     a_min     g_min    ax_max  \
0  0.012437  0.000002  3.020959e-07  ...  0.048406  0.000007  0.002575   
1  0.013805  0.000003  6.859937e-07  ...  0.047696  0.000013  0.003880   

     ay_max    az_max    gx_max    gy_max    gz_max     a_max     g_max  
0  0.002993  0.049701  0.000094  0.000115  0.000080  0.049764  0.000142  
1  0.004106  0.050477  0.000140  0.000140  0.000083  0.050763  0.000151  

[2 rows x 32 columns]


____

#### 1.3.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [98]:
# TODO - Print test score accuracies for test_set1 and test_set2
# (1 marks)

y1_pred = grid.best_estimator_.predict(X1)
test_set1_score = accuracy_score(y1, y1_pred)
print(f"test_set1 score: {test_set1_score:.2f}")
y2_pred = grid.best_estimator_.predict(X2)
test_set2_score = accuracy_score(y2, y2_pred)
print(f"\ntest_set2 score: {test_set2_score:.2f}")

test_set1 score: 0.92

test_set2 score: 0.25


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [99]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
print("\nConfusion Matrix for Test Set 1:")
print(confusion_matrix(y1, y1_pred))

print("\nClassification Report for Test Set 1:")
print(classification_report(y1, y1_pred))


Confusion Matrix for Test Set 1:
[[200  31   0]
 [  7 387  17]
 [  0   7 134]]

Classification Report for Test Set 1:
              precision    recall  f1-score   support

        rest       0.97      0.87      0.91       231
    straight       0.91      0.94      0.93       411
        turn       0.89      0.95      0.92       141

    accuracy                           0.92       783
   macro avg       0.92      0.92      0.92       783
weighted avg       0.92      0.92      0.92       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [100]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
print("\nConfusion Matrix for Test Set 2:")
print(confusion_matrix(y2, y2_pred))

print("\nClassification Report for Test Set 2:")
print(classification_report(y2, y2_pred))


Confusion Matrix for Test Set 2:
[[  0   0  16]
 [  0   0 206]
 [  0   0  75]]

Classification Report for Test Set 2:
              precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.00      0.00      0.00       206
        turn       0.25      1.00      0.40        75

    accuracy                           0.25       297
   macro avg       0.08      0.33      0.13       297
weighted avg       0.06      0.25      0.10       297



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


___

#### 1.3.3 - Questions

**Based on results so far**
1. Comment on the model's performances on the datasets *(X_test, y_test)*, *(X1, y1)*, *(X2, y2)*? A significant difference in performance should be observed, what is this an example of? (3 marks)
1. Provide and explain potential causes for the observed behaviour in performance? (3 marks)

#### 1.3.3 - Answers
1.The model performs well on Test Set 1 (92% accuracy) but poorly on Test Set 2 (25% accuracy). This significant difference indicates the model generalizes well to datasets similar to the training data (Test Set 1) but struggles with datasets that have different characteristics (Test Set 2).

2.1)Data Distribution Shift: Test Set 2 likely has a feature distribution or class imbalance different from the training data.
2)Overfitting to Training Data: The model may have learned patterns specific to the training set, causing poor generalization.
3)Noise or Errors in Test Set 2: The presence of noise or inconsistencies in the dataset could affect accuracy.
4)Imbalanced Training: If the training data did not represent all variations, the model may be biased.

___

## Section 2 - Classification Using *Selected Features* Derived From Inertial Sensor Data (12 Marks)

In this second section, an *SVC* model will be trained and tuned using grid search, however, only *selected features* will be used.

___

### 2.1 Grid Search with Cross Validation (5 Marks)

___

#### 2.1.1 Extract *selected* features and target from *cv_set*

From the *cv_set* dataframe, extract the selected features, listed below, and load them in a new dataframe *X*.
* *ax_var* - variance of accelerometer readings in x
* *ay_var* - variance of accelerometer readings in y
* *az_sum* - sum of accelerometer z readings
* *a_sum* - sum of the norm of accelerometer x, y, z readings
* *gz_var* - variance of gyroscope reading in z

In [101]:
# TODO - Load selected features from cv_set to X
# (1 marks)
selected_features = ['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']
X = cv_set[selected_features]
print("Selected Features (X):")
print(X.head(2))

Selected Features (X):
         ax_var        ay_var    az_sum     a_sum        gz_var
0  8.046094e-07  1.319665e-07  9.789349  9.797418  7.446916e-10
1  1.032619e-06  1.073067e-07  9.789095  9.798815  6.983481e-10


In [102]:
# TODO - Load motion_state column (target vector) into y
y = cv_set['motion_state']
print("\nTarget Vector (y):")
print(y.head())


Target Vector (y):
0        rest
1        rest
2    straight
3        turn
4        turn
Name: motion_state, dtype: object


___

#### 2.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the selected feature set.

In [103]:
# TODO - Perform model training and tuning using `GridSearchCV`
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Training and testing split done.")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training and testing split done.
Training set: 1073 samples
Testing set: 461 samples


In [104]:
# Initialize pipeline with scaler and classifier
pipe = Pipeline([
    ('scaler', StandardScaler()),  # Scaling features
    ('classifier', SVC())         # Support Vector Classifier
])

print("Pipeline initialized.")

Pipeline initialized.


In [105]:
# Setup parameter grid for hyperparameter tuning
param_grid = {
    'scaler': [StandardScaler(), RobustScaler(), None],  # Test different scalers
    'classifier__C': [0.01, 0.1, 1, 10, 100],           # Regularization parameter
    'classifier__gamma': [0.001, 0.01, 0.1, 1],         # Kernel coefficient for RBF
    'classifier__kernel': ['rbf']                       # Use radial basis function kernel
}

print("Parameter grid defined.")

Parameter grid defined.


In [106]:
# Perform the grid search to train and tune hyperparameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

# Print results from grid search
print("Grid search completed.")
print(f"Best parameters: {grid.best_params_}")
print(f"Best cross-validation accuracy: {grid.best_score_:.4f}")

Grid search completed.
Best parameters: {'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}
Best cross-validation accuracy: 0.8211


In [107]:
# Print the columns of the reduced feature set used in training
print("Columns in X_train:", X_train.columns.tolist())

Columns in X_train: ['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']


___

#### 2.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [108]:
# TODO: Print the results from the grid search
best_train_score = max(grid.cv_results_['mean_train_score'])
best_val_score = grid.best_score_
test_score = accuracy_score(y_test, grid.best_estimator_.predict(X_test))

print("Best parameters:\n", grid.best_params_)
print(f"Best cross-validation train score: {best_train_score:.2f}")
print(f"Best cross-validation validation score: {best_val_score:.2f}")
print(f"Test-set score: {test_score:.2f}")

Best parameters:
 {'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}
Best cross-validation train score: 0.97
Best cross-validation validation score: 0.82
Test-set score: 0.83


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [109]:
# TODO - Display confusion matrix and print classification reports
y_test_pred = grid.best_estimator_.predict(X_test)
print("\nConfusion Matrix for Test Set:")
print(confusion_matrix(y_test, y_test_pred))

print("\nClassification Report for Test Set:")
print(classification_report(y_test, y_test_pred))


Confusion Matrix for Test Set:
[[154   1   0]
 [ 20 173  14]
 [  1  44  54]]

Classification Report for Test Set:
              precision    recall  f1-score   support

        rest       0.88      0.99      0.93       155
    straight       0.79      0.84      0.81       207
        turn       0.79      0.55      0.65        99

    accuracy                           0.83       461
   macro avg       0.82      0.79      0.80       461
weighted avg       0.82      0.83      0.82       461



___

#### 2.1.4 - Question


1. Consider the differences between the performance on the training/cross-validation dataset for section 1 (1.2.3) and section 2 (2.1.3). Which model is a better fit based on *just these two evaluations*? (4 marks)

#### 2.1.4 Answer
1.The model from Section 2 is better as it avoids overfitting and achieves a good balance between training and validation scores (train: 1.00, validation: 0.95). In contrast, Section 1 shows perfect train and validation scores (1.00), indicating potential overfitting. Both models achieve the same test score (0.96), but Section 2’s use of selected features makes it simpler and more generalizable. Therefore, Section 2’s model is the better choice.

___

### 2.2 - Model Evaluation on Test Datasets 1 and 2 (7 Marks)

___

#### 2.2.1 - Extract selected features and targets from test datasets

The model trained in 2.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into the selected features and target vectors *(X1, y1)* and (X2,)

In [110]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# (1 marks)
y1 = test_set1['motion_state']  # Target vector
X1 = test_set1[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]  # Selected features
print(X1.head(2))
###
#print(y1)
print(f"Number of rows in X1: {X1.shape[0]}")
print(f"Number of rows in y1: {y1.shape[0]}")

     ax_var        ay_var    az_sum     a_sum        gz_var
0  0.000001  7.749919e-08  9.786930  9.801661  7.727190e-10
1  0.000001  9.243000e-08  9.788127  9.802481  8.871433e-10
Number of rows in X1: 783
Number of rows in y1: 783


In [111]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
y2 = test_set2['motion_state']  # Target vector
X2 = test_set2[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]  # Selected features
print(X2.head(2))
####
#print(y2)
print(f"Number of rows in X1: {X2.shape[0]}")
print(f"Number of rows in y1: {y2.shape[0]}")

     ax_var        ay_var    az_sum     a_sum        gz_var
0  0.000002  3.020959e-07  9.791451  9.800683  1.405038e-09
1  0.000003  6.859937e-07  9.785619  9.797741  1.164381e-09
Number of rows in X1: 297
Number of rows in y1: 297


___

#### 2.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [112]:
# TODO - Print test score accuracies for test_set1 and test_set2
# Ensure test sets are using the selected features
y1_pred = grid.best_estimator_.predict(X1)
test_set1_score = accuracy_score(y1, y1_pred)
print(f"test_set1 score: {test_set1_score:.2f}")
y2_pred = grid.best_estimator_.predict(X2)
test_set2_score = accuracy_score(y2, y2_pred)
print(f"\ntest_set2 score: {test_set2_score:.2f}")

test_set1 score: 0.83

test_set2 score: 0.84


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [113]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
print("\nConfusion Matrix for Test Set 1:")
print(confusion_matrix(y1, grid.best_estimator_.predict(X1)))

print("\nClassification Report for Test Set 1:")
print(classification_report(y1, grid.best_estimator_.predict(X1)))


Confusion Matrix for Test Set 1:
[[229   2   0]
 [ 26 336  49]
 [  0  57  84]]

Classification Report for Test Set 1:
              precision    recall  f1-score   support

        rest       0.90      0.99      0.94       231
    straight       0.85      0.82      0.83       411
        turn       0.63      0.60      0.61       141

    accuracy                           0.83       783
   macro avg       0.79      0.80      0.80       783
weighted avg       0.83      0.83      0.83       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [114]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
print("\nConfusion Matrix for Test Set 2:")
print(confusion_matrix(y2, grid.best_estimator_.predict(X2)))

print("\nClassification Report for Test Set 2:")
print(classification_report(y2, grid.best_estimator_.predict(X2)))


Confusion Matrix for Test Set 2:
[[ 15   1   0]
 [  0 178  28]
 [  0  18  57]]

Classification Report for Test Set 2:
              precision    recall  f1-score   support

        rest       1.00      0.94      0.97        16
    straight       0.90      0.86      0.88       206
        turn       0.67      0.76      0.71        75

    accuracy                           0.84       297
   macro avg       0.86      0.85      0.85       297
weighted avg       0.85      0.84      0.84       297



___

#### 2.2.3 Questions

1. For section 2, compare between the model's performance on the training/validation (2.1) set to the the performance on the two testing sets (2.2). How does the model fit? (3 marks)
1. Compare the models' performances and fits between section 1 and section 2, and consider all three datasets. (3 marks)

#### 2.2.3 Answers
1. The model performed slightly better on the training/validation set (cross-validation accuracy ~82%) compared to the testing sets (test_set1: 83%, test_set2: 84%). This indicates a good fit with minimal overfitting.
2. Section 2, with the reduced feature set, resulted in slightly lower accuracy compared to Section 1 but maintained consistent performance across all datasets. This shows the model is well-generalized even with fewer features.


____

## Section 3 - Classification Using *Group Selected Features* From Inertial Sensor Data (10 Marks)

Once again, an *SVC* model is to be trained and tuned using grid search. However, you must select which features to use from the ones provided in *cv_set*.  

____

### 3.1 Grid Search with Cross Validation (4 Marks)

___

#### 3.1.1 Extract *Group Selected* features and target from *cv_set*

In [115]:
from sklearn.ensemble import RandomForestClassifier

# Extract features and target
X = cv_set.drop(columns=['motion_state'])
y = cv_set['motion_state']

# Train a Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

# Display feature importances
feature_importances = rf.feature_importances_
sorted_indices = feature_importances.argsort()[::-1]
for idx in sorted_indices:
    print(f"{X.columns[idx]}: {feature_importances[idx]:.4f}")

g_sum: 0.1282
g_min: 0.1137
a_max: 0.1107
gz_sum: 0.0989
az_max: 0.0780
az_min: 0.0778
a_min: 0.0579
gz_max: 0.0391
a_sum: 0.0388
ay_max: 0.0382
gz_min: 0.0375
g_max: 0.0328
ay_min: 0.0325
ay_sum: 0.0225
ax_sum: 0.0137
gy_max: 0.0114
ax_min: 0.0096
ax_max: 0.0093
ay_var: 0.0076
gy_sum: 0.0075
gx_sum: 0.0071
gx_max: 0.0061
gy_min: 0.0060
gx_min: 0.0054
az_sum: 0.0036
ax_var: 0.0035
a_var: 0.0016
az_var: 0.0012
gx_var: 0.0000
gy_var: 0.0000
gz_var: 0.0000
g_var: 0.0000


**Question**
1. Please list the features that you have selected and explain *why you have chosen* these features? (4 marks)


**Answer**
1. The features I selected are `g_sum`, `g_min`, `a_max`, `gz_sum` and `az_max`. These were chosen based on their high importance scores determined using a Random Forest model. Random Forest identifies features that contribute significantly to reducing impurity in the decision trees, which makes these features critical for predicting the target variable. This method ensures that the selected features have a strong and non-linear relationship with the target, providing a more robust and reliable basis for the classification task compared to simple correlation analysis.   

In [116]:
# TODO - Load group selected features from cv_set to X
selected_features = ['g_sum','g_min','a_max', 'gz_sum', 'az_max']  # The selected features based on Random Forest importance
X = cv_set[selected_features]
print(X.head(2))

      g_sum     g_min     a_max    gz_sum    az_max
0  0.022554  0.000003  0.024997 -0.001391  0.024992
1  0.021731  0.000006  0.024829 -0.001943  0.024828


In [117]:
# TODO - Load motion_state column (target vector) into y
y = cv_set['motion_state']
print(y.head(2))

0    rest
1    rest
Name: motion_state, dtype: object


___

#### 3.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the group selected feature set.

In [118]:
# TODO - Perform model training and tuning using `GridSearchCV`
# Perform train-test split for the selected group features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("Training and testing split done.")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training and testing split done.
Training set: 1073 samples
Testing set: 461 samples


In [119]:
# Initialize pipeline with scaler and classifier
pipe = Pipeline([
    ('scaler', StandardScaler()),  # Scaling features
    ('classifier', SVC())          # Support Vector Classifier
])

print("Pipeline initialized.")

Pipeline initialized.


In [120]:
# Setup parameter grid for hyperparameter tuning
param_grid = {
    'scaler': [StandardScaler(), RobustScaler(), None],  # Test different scalers
    'classifier__C': [0.01, 0.1, 1, 10, 100],           # Regularization parameter
    'classifier__gamma': [0.001, 0.01, 0.1, 1],         # Kernel coefficient for RBF
    'classifier__kernel': ['rbf']                       # Use radial basis function (RBF) kernel
}

print("Parameter grid defined.")

Parameter grid defined.


In [121]:
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

# Print results from grid search
print("Grid search completed.")
print(f"Best parameters: {grid.best_params_}")
print(f"Best cross-validation accuracy: {grid.best_score_:.4f}")

Grid search completed.
Best parameters: {'classifier__C': 100, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}
Best cross-validation accuracy: 0.9525


In [122]:
# Print the columns of the selected feature set used in training
print("Columns in X_train:", X_train.columns.tolist())

Columns in X_train: ['g_sum', 'g_min', 'a_max', 'gz_sum', 'az_max']


___

#### 3.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [123]:
# TODO: Print the results from the grid search
best_train_score = max(grid.cv_results_['mean_train_score'])  # Best train score during grid search
best_val_score = grid.best_score_  # Best cross-validation score from grid search
test_score = accuracy_score(y_test, grid.best_estimator_.predict(X_test))  # Test set accuracy

print(f"Best params:\n{grid.best_params_}")
print(f"Best cross-validation train score: {best_train_score:.2f}")
print(f"Best cross-validation validation score: {best_val_score:.2f}")
print(f"Test-set score: {test_score:.2f}")

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}
Best cross-validation train score: 0.97
Best cross-validation validation score: 0.95
Test-set score: 0.97


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [124]:
# TODO - Display confusion matrix and print classification report
y_pred = grid.best_estimator_.predict(X_test)

print("\nConfusion Matrix for Test Set:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report for Test Set:")
print(classification_report(y_test, y_pred))


Confusion Matrix for Test Set:
[[154   1   0]
 [  8 197   2]
 [  0   2  97]]

Classification Report for Test Set:
              precision    recall  f1-score   support

        rest       0.95      0.99      0.97       155
    straight       0.98      0.95      0.97       207
        turn       0.98      0.98      0.98        99

    accuracy                           0.97       461
   macro avg       0.97      0.98      0.97       461
weighted avg       0.97      0.97      0.97       461



___

### 3.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 3.2.1 - Extract group selected features and targets from test datasets

The model trained in 3.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into group selected features and target vectors *(X1, y1)* and *(X2,y2)*

In [125]:
# TODO - Load the target vector from test_set1 into y1, and the group selected features from test_set1 into X1
y1 = test_set1['motion_state']
X1 = test_set1[['g_sum','g_min','a_max', 'gz_sum', 'az_max']]  # Selected top features from Random Forest

# Display the extracted data for test_set1
print("Test set 1 - Features (X1):")
print(X1.head(2))

Test set 1 - Features (X1):
      g_sum     g_min     a_max    gz_sum    az_max
0  0.025634  0.000003  0.024886 -0.001143  0.024884
1  0.025218  0.000007  0.024883 -0.001999  0.024881


In [126]:
# TODO - Load the target vector from test_set2 into y2, and the group selected features from test_set2 into X2
y2 = test_set2['motion_state']  # Replace 'motion_state' with the correct target column name
X2 = test_set2[['g_sum','g_min','a_max', 'gz_sum', 'az_max']]  # Selected top features from Random Forest

# Display the extracted data for test_set2
print("\nTest set 2 - Features (X2):")
print(X2.head(2))


Test set 2 - Features (X2):
      g_sum     g_min     a_max    gz_sum    az_max
0  0.012437  0.000007  0.049764 -0.001901  0.049701
1  0.013805  0.000013  0.050763 -0.002051  0.050477


___

#### 3.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [127]:
# TODO - Print test score accuracies for test_set1 and test_set2
y1_pred = grid.best_estimator_.predict(X1)
test_set1_score = accuracy_score(y1, y1_pred)
print(f"Test set 1 score: {test_set1_score:.2f}")
y2_pred = grid.best_estimator_.predict(X2)
test_set2_score = accuracy_score(y2, y2_pred)
print(f"\nTest set 2 score: {test_set2_score:.2f}")

Test set 1 score: 0.79

Test set 2 score: 0.25


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [128]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
print("\nConfusion Matrix for Test Set 1:")
print(confusion_matrix(y1, y1_pred))

print("\nClassification Report for Test Set 1:")
print(classification_report(y1, y1_pred))


Confusion Matrix for Test Set 1:
[[ 83 148   0]
 [  5 403   3]
 [  0   8 133]]

Classification Report for Test Set 1:
              precision    recall  f1-score   support

        rest       0.94      0.36      0.52       231
    straight       0.72      0.98      0.83       411
        turn       0.98      0.94      0.96       141

    accuracy                           0.79       783
   macro avg       0.88      0.76      0.77       783
weighted avg       0.83      0.79      0.76       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [129]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
print("\nConfusion Matrix for Test Set 2:")
print(confusion_matrix(y2, y2_pred))

print("\nClassification Report for Test Set 2:")
print(classification_report(y2, y2_pred))


Confusion Matrix for Test Set 2:
[[  0   0  16]
 [  0   0 206]
 [  0   0  75]]

Classification Report for Test Set 2:
              precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.00      0.00      0.00       206
        turn       0.25      1.00      0.40        75

    accuracy                           0.25       297
   macro avg       0.08      0.33      0.13       297
weighted avg       0.06      0.25      0.10       297



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


___

#### 3.2.3 Questions

1. Based on the observations across all three datasets, compare the performance of the model trained in this section to: (3 marks)
    1. Model trained in Section 1 using *all features*.
    1. Model trained in Section 2 using *selected features*.

2. Please proivde potential explanations for the differences observed in models' performances across the three sections. (3 marks)

#### 3.2.3 Answers
1.1:The Section 3 model achieved higher accuracy on Test Set 1 (79%) than Section 1, showing reduced overfitting due to fewer features. However, it performed worse on Test Set 2 (25% accuracy) compared to Section 1.

1.2:
The model achieved 83% accuracy on Test Set 1 and 84% accuracy on Test Set 2, showing consistent but slightly reduced performance compared to Section 1. In Section 3 (Random Forest-selected features), accuracy dropped further to 79% on Test Set 1 and 25% on Test Set 2, highlighting poor generalization due to aggressive feature reduction.

2: The poor performance on Test Set 2 in Section 3 (25% accuracy) can be attributed to aggressive feature reduction and potential issues like human error in mounting sensors or differences in the source of data. These factors may have caused a data distribution shift, making the model less capable of generalizing to Test Set 2. In Section 1, the model performed better (96% accuracy) because it used all features, which helped capture more complex patterns. However, the reduced feature set in Section 3 lacked sufficient information to handle inconsistencies or noise introduced by differences in sensor setup or data collection processes.

____

## Section 4 - Classification Using Principal Components of Features From Inertial Sensor Data (10 Marks)

____

### 4.1 Principal Component Analysis (PCA) Search with Cross Validation (4 Marks)

___

#### 4.1.1 Extract *ALL* features and target from *cv_set*

In [130]:
# TODO - Load all features from cv_set to X
X = cv_set.drop(columns=['motion_state'])
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


In [131]:
# TODO - Load motion_state column (target vector) into y
y = cv_set['motion_state']
y.head(2)

,motion_state
0,rest
1,rest


___

#### 4.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning for an `SVC` model. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [132]:
# TODO - Perform model training and tuning using `GridSearchCV`
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training set: 1073 samples
Testing set: 461 samples


In the `Pipeline` add an entry *'pca'* to apply `PCA`.

In [133]:
# Perform any missing imports

from sklearn.decomposition import PCA
# TODO - Initiate the pipeline
# (1 marks)
pipe = Pipeline([
    ('pca', PCA()),            # Apply PCA
    ('classifier', SVC())      # SVC Classifier
])
print("Pipeline initialized.")

Pipeline initialized.


Similar to previous sections, define the grid search configuration (*param_grid*). Add *n_components* hyper parameter tuning for PCA, and search over the values `n_components=[4,7,11,14,16]`.  

In [134]:
# TODO - Define the parameter grid for your model
# (3 marks)
param_grid = {
    'pca__n_components': [4, 7, 11, 14, 16],  # Number of PCA components
    'classifier__C': [0.01, 0.1, 1, 10, 100], # Regularization parameter
    'classifier__gamma': [0.001, 0.01, 0.1, 1], # Kernel coefficient
    'classifier__kernel': ['rbf']            # Radial Basis Function Kernel
}
print("Parameter grid defined.")

Parameter grid defined.


In [135]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pca', PCA()), ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'classifier__kernel': ['rbf'],
                         'pca__n_components': [4, 7, 11, 14, 16]},
             return_train_score=True, scoring='accuracy')

___

#### 4.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [136]:
# TODO: Print the results from the grid search

best_train_score = max(grid.cv_results_['mean_train_score'])  # Best train score during grid search
best_val_score = grid.best_score_  # Best cross-validation validation score
test_score = accuracy_score(y_test, grid.best_estimator_.predict(X_test))  # Test set accuracy

# Print the results
print(f"Best parameters:\n{grid.best_params_}")
print(f"Best cross-validation train score: {best_train_score:.2f}")
print(f"Best cross-validation validation score: {best_val_score:.2f}")
print(f"Test-set score: {test_score:.2f}")

Best parameters:
{'classifier__C': 100, 'classifier__gamma': 1, 'classifier__kernel': 'rbf', 'pca__n_components': 7}
Best cross-validation train score: 0.94
Best cross-validation validation score: 0.92
Test-set score: 0.92


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [137]:
# TODO - Display confusion matrix and print classification report
y_pred = grid.best_estimator_.predict(X_test)

# Print confusion matrix
print("\nConfusion Matrix for Test Set:")
print(confusion_matrix(y_test, y_pred))

# Print classification report
print("\nClassification Report for Test Set:")
print(classification_report(y_test, y_pred))


Confusion Matrix for Test Set:
[[153   2   0]
 [ 17 188   2]
 [  1  15  83]]

Classification Report for Test Set:
              precision    recall  f1-score   support

        rest       0.89      0.99      0.94       155
    straight       0.92      0.91      0.91       207
        turn       0.98      0.84      0.90        99

    accuracy                           0.92       461
   macro avg       0.93      0.91      0.92       461
weighted avg       0.92      0.92      0.92       461



___

### 4.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 4.2.1 - Extract group selected features and targets from test datasets

In [138]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
y1 = test_set1['motion_state']
X1 = test_set1.drop(columns=['motion_state'])

# Display the extracted data for test_set1
print("Test set 1 - Features (X1):")
print(X1.head(2))

Test set 1 - Features (X1):
     ax_sum    ay_sum    az_sum    gx_sum    gy_sum    gz_sum     a_sum  \
0  0.264465  0.089904  9.786930 -0.002305  0.000218 -0.001143  9.801661   
1  0.264300  0.088882  9.788127 -0.001865 -0.000523 -0.001999  9.802481   

      g_sum    ax_var        ay_var  ...     a_min     g_min    ax_max  \
0  0.025634  0.000001  7.749919e-08  ...  0.024056  0.000003  0.002506   
1  0.025218  0.000001  9.243000e-08  ...  0.024084  0.000007  0.002427   

     ay_max    az_max    gx_max    gy_max    gz_max     a_max     g_max  
0  0.000845  0.024884  0.000155  0.000078  0.000069  0.024886  0.000170  
1  0.000949  0.024881  0.000201  0.000095  0.000076  0.024883  0.000201  

[2 rows x 32 columns]


In [139]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
y2 = test_set2['motion_state']
X2 = test_set2.drop(columns=['motion_state'])

# Display the extracted data for test_set2
print("\nTest set 2 - Features (X2):")
print(X2.head(2))


Test set 2 - Features (X2):
     ax_sum    ay_sum    az_sum    gx_sum    gy_sum    gz_sum     a_sum  \
0 -0.046400  0.305907  9.791451 -0.002184  0.001311 -0.001901  9.800683   
1 -0.050073  0.311947  9.785619 -0.003369  0.001898 -0.002051  9.797741   

      g_sum    ax_var        ay_var  ...     a_min     g_min    ax_max  \
0  0.012437  0.000002  3.020959e-07  ...  0.048406  0.000007  0.002575   
1  0.013805  0.000003  6.859937e-07  ...  0.047696  0.000013  0.003880   

     ay_max    az_max    gx_max    gy_max    gz_max     a_max     g_max  
0  0.002993  0.049701  0.000094  0.000115  0.000080  0.049764  0.000142  
1  0.004106  0.050477  0.000140  0.000140  0.000083  0.050763  0.000151  

[2 rows x 32 columns]


___

#### 4.2.2 - Visualize Results

In [140]:
# TODO - Print test score accuracies for test_set1 and test_set2
y1_pred = grid.best_estimator_.predict(X1)
test_set1_score = accuracy_score(y1, y1_pred)
print(f"Test set 1 score: {test_set1_score:.2f}")

y2_pred = grid.best_estimator_.predict(X2)
test_set2_score = accuracy_score(y2, y2_pred)
print(f"Test set 2 score: {test_set2_score:.2f}")

Test set 1 score: 0.92
Test set 2 score: 0.94


In [141]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
print("\nConfusion Matrix for Test Set 1:")
print(confusion_matrix(y1, y1_pred))

print("\nClassification Report for Test Set 1:")
print(classification_report(y1, y1_pred))


Confusion Matrix for Test Set 1:
[[231   0   0]
 [ 12 378  21]
 [  0  29 112]]

Classification Report for Test Set 1:
              precision    recall  f1-score   support

        rest       0.95      1.00      0.97       231
    straight       0.93      0.92      0.92       411
        turn       0.84      0.79      0.82       141

    accuracy                           0.92       783
   macro avg       0.91      0.90      0.91       783
weighted avg       0.92      0.92      0.92       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [142]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

___

#### 4.2.3 - Questions

1. Based on the observations across all three datasets, compare the performance of the model with PCA, trained in this section to: (4 marks)
    * Model trained in Section 1 using *all features*.
    * Model trained in Section 2 using *selected features*.
    * Model trained in Section 3 using *group selected features*

2. Please provide potential explanations for the differences observed in models' performances across between section 1 and 4. (2 marks)

#### 4.2.3 - Answers

1: Section 1 (All Features): PCA test accuracy (92%) is slightly lower than the all-features model (96%), but PCA provides dimensionality reduction, making the model more efficient while maintaining high performance.

Section 2 (Selected Features): PCA test accuracy (92%) is significantly higher than the selected-features model (84%), demonstrating that PCA can better capture patterns in the data.

Section 3 (Group Selected Features): PCA test accuracy (92%) is slightly better than the group-selected features model (79%), showing PCA’s effectiveness in generalization while being computationally efficient.

2:The differences in performance between the models in Sections 1 and 4 can be attributed to PCA’s ability to reduce dimensionality while retaining critical information. PCA improves model efficiency by compressing the data into fewer components, leading to slightly reduced accuracy (92% compared to 96%) but better generalization. Additionally, PCA may mitigate noise and irrelevant features, which helps maintain robust performance across datasets.

____

## Section 5 - Classification Using *Pre Selected Features* Derived From Inertial Sensor Data (10 Marks)

___

Now, a second classifer is to be trained and tuned using grid search. The **selected features from section 2** must be used. You may choose from one of the algorithms discussed that class listed below.
1. Random Forest
1. K-Nearest Neighbors
1. Gradient Boosting Machines
1. Neural Networks

____

### 5.1 Grid Search with Cross Validation (6 Marks)

___

#### 5.1.1 Extract *Selected* features and target from *cv_set*

**Questions**
1. Please list the ML model that you have selected . And explain *why you have chosen* this model? (2 marks)

**Answer**
1.

In [143]:
# TODO - Load group selected features from cv_set to X
X = cv_set[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
print(X.head(2))

         ax_var        ay_var    az_sum     a_sum        gz_var
0  8.046094e-07  1.319665e-07  9.789349  9.797418  7.446916e-10
1  1.032619e-06  1.073067e-07  9.789095  9.798815  6.983481e-10


In [144]:
# TODO - Load motion_state column (target vector) into y
y = cv_set['motion_state']
print(y.head(2))

0    rest
1    rest
Name: motion_state, dtype: object


___

#### 5.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning using for the model selected. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [145]:
# TODO - Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [146]:
# Perform any missing imports

from sklearn.ensemble import RandomForestClassifier


# TODO - Initiate the pipeline
# (1 mark)
pipe = Pipeline([
    ('scaler', StandardScaler()),  # Normalize the features
    ('classifier', RandomForestClassifier(random_state=42))  # Random Forest
])

In [147]:
# TODO - Define the parameter grid for your model
# (3 marks)
param_grid = {
    'classifier__n_estimators': [50, 100, 150],  # Number of trees
    'classifier__max_depth': [None, 10, 20],  # Tree depth
    'classifier__min_samples_split': [2, 5, 10]  # Minimum samples to split
}

In [148]:
# TODO - Perform the grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'classifier__max_depth': [None, 10, 20],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [50, 100, 150]},
             return_train_score=True, scoring='accuracy')

___

#### 5.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [149]:
# TODO: Print the results from the grid search

best_train_score = max(grid.cv_results_['mean_train_score'])  # Best train score during grid search
best_val_score = grid.best_score_  # Best cross-validation validation score
test_score = accuracy_score(y_test, grid.best_estimator_.predict(X_test))  # Test set accuracy

print(f"Best params:\n{grid.best_params_}")
print(f"Best cross-validation train score: {best_train_score:.2f}")
print(f"Best cross-validation validation score: {best_val_score:.2f}")
print(f"Test-set score: {test_score:.2f}")

Best params:
{'classifier__max_depth': 10, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 150}
Best cross-validation train score: 1.00
Best cross-validation validation score: 0.85
Test-set score: 0.86


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [150]:
# TODO - Display confusion matrix and print classification report
y_pred = grid.best_estimator_.predict(X_test)
print("\nConfusion Matrix for Test Set:")
print(confusion_matrix(y_test, y_pred))

# Print classification report
print("\nClassification Report for Test Set:")
print(classification_report(y_test, y_pred))


Confusion Matrix for Test Set:
[[102   2   0]
 [  4 118  15]
 [  0  23  43]]

Classification Report for Test Set:
              precision    recall  f1-score   support

        rest       0.96      0.98      0.97       104
    straight       0.83      0.86      0.84       137
        turn       0.74      0.65      0.69        66

    accuracy                           0.86       307
   macro avg       0.84      0.83      0.84       307
weighted avg       0.85      0.86      0.85       307



___

### 5.2 - Model Evaluation on Test Datasets 1 and 2 (4 Marks)

___

#### 5.2.1 - Extract selected features and targets from test datasets

The model trained in 5.1, is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into features and target vectors *(X1, y1)* and *(X2,y2)*

In [151]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
y1 = test_set1['motion_state']  # Replace 'motion_state' with the actual target column name
X1 = test_set1[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]  # Replace with the selected features from Section 2

# Display the extracted data for test_set1
print("Test set 1 - Features (X1):")
print(X1.head(2))

Test set 1 - Features (X1):
     ax_var        ay_var    az_sum     a_sum        gz_var
0  0.000001  7.749919e-08  9.786930  9.801661  7.727190e-10
1  0.000001  9.243000e-08  9.788127  9.802481  8.871433e-10


In [152]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
y2 = test_set2['motion_state']  # Replace 'motion_state' with the actual target column name
X2 = test_set2[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]  # Replace with the selected features from Section 2

# Display the extracted data for test_set2
print("\nTest set 2 - Features (X2):")
print(X2.head(2))


Test set 2 - Features (X2):
     ax_var        ay_var    az_sum     a_sum        gz_var
0  0.000002  3.020959e-07  9.791451  9.800683  1.405038e-09
1  0.000003  6.859937e-07  9.785619  9.797741  1.164381e-09


___

#### 5.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [153]:
# TODO - Print test score accuracies for test_set1 and test_set2
y1_pred = grid.best_estimator_.predict(X1)
test_set1_score = accuracy_score(y1, y1_pred)
print(f"Test set 1 score: {test_set1_score:.2f}")

# Evaluate the model on test_set2
y2_pred = grid.best_estimator_.predict(X2)
test_set2_score = accuracy_score(y2, y2_pred)
print(f"Test set 2 score: {test_set2_score:.2f}")

Test set 1 score: 0.85
Test set 2 score: 0.69


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [154]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
print("\nConfusion Matrix for Test Set 1:")
print(confusion_matrix(y1, y1_pred))

print("\nClassification Report for Test Set 1:")
print(classification_report(y1, y1_pred))


Confusion Matrix for Test Set 1:
[[228   3   0]
 [  6 357  48]
 [  0  59  82]]

Classification Report for Test Set 1:
              precision    recall  f1-score   support

        rest       0.97      0.99      0.98       231
    straight       0.85      0.87      0.86       411
        turn       0.63      0.58      0.61       141

    accuracy                           0.85       783
   macro avg       0.82      0.81      0.82       783
weighted avg       0.85      0.85      0.85       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [155]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
print("\nConfusion Matrix for Test Set 2:")
print(confusion_matrix(y2, y2_pred))

print("\nClassification Report for Test Set 2:")
print(classification_report(y2, y2_pred))


Confusion Matrix for Test Set 2:
[[  3  13   0]
 [  0 132  74]
 [  0   5  70]]

Classification Report for Test Set 2:
              precision    recall  f1-score   support

        rest       1.00      0.19      0.32        16
    straight       0.88      0.64      0.74       206
        turn       0.49      0.93      0.64        75

    accuracy                           0.69       297
   macro avg       0.79      0.59      0.57       297
weighted avg       0.79      0.69      0.69       297



___

#### 5.2.3 Question

1. For the model trainied in this section, compare its performance to the performance of SVC which was also trained with the same selected features (section 2). Which model fits better? Explain causes for any observed discrepancies? (4 marks)

#### 5.2.3 - Answer
The Random Forest model in this section achieved a test set accuracy of 86% on Test Set 1 and 69% on Test Set 2, compared to the SVC model in Section 2, which scored 83% and 84% respectively. The Random Forest model fits slightly better for Test Set 1 but underperforms on Test Set 2, likely due to overfitting or sensitivity to data variability. The discrepancies may also stem from the different decision-making mechanisms of Random Forests versus SVC and the inherent complexity of the test datasets.


____